<a href="https://colab.research.google.com/github/ErikSeras/usos_gee/blob/master/analisis_atmosfera/gif_goes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Cómo poder visualizar un huracán?

In [2]:
## Instalar geemap
!pip install geemap

     |████████████████████████████████| 378kB 2.7MB/s 
     |████████████████████████████████| 4.7MB 8.3MB/s 
     |████████████████████████████████| 6.9MB 34.4MB/s 
     |████████████████████████████████| 1.1MB 44.5MB/s 
     |████████████████████████████████| 5.1MB 44.5MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 225kB 47.0MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 552kB 42.9MB/s 
     |████████████████████████████████| 215kB 39.8MB/s 
     |████████████████████████████████| 112kB 48.4MB/s 
     |████████████████████████████████| 122kB 48.5MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=b32c99400b42fb520723f615a1dd03fa512e964c4d40d3765d154977d0de4d40
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
  Created wheel for i

In [3]:
## Librerías necesarias
import ee
import geemap.eefolium as geemap

In [ ]:
# Iniciar sesión de GEE
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [5]:
# Importar el gif
! wget https://drive.google.com/file/d/1mrFQ3ssm9wAhoJgYYoTNK1orN3fkY6BU/view?usp=sharing

--2020-09-30 09:39:22--  https://drive.google.com/file/d/1mrFQ3ssm9wAhoJgYYoTNK1orN3fkY6BU/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 173.194.217.113, 173.194.217.139, 173.194.217.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  69.38K  --.-KB/s    in 0.001s  

2020-09-30 09:39:22 (50.4 MB/s) - ‘view?usp=sharing’ saved [71041]



In [ ]:
saved_gif = os.path.abspath("../input/katrina_2005_08_28.gif")
geemap.show_image(saved_gif)

In [ ]:
## Dirección del GIF a modificar
in_gif = os.path.abspath('../input/katrina_2005_08_28.gif')

## Dirección del GIF modificado
out_gif = os.path.abspath('../output/huracan_katrina_texto.gif')

In [ ]:
geemap.add_text_to_gif(in_gif, out_gif, xy=('3%', '90%'),
                       text_sequence="Fecha de inicio - fin de la imagen",
                       font_size=15, font_color='white', duration=70)

In [ ]:
geemap.add_text_to_gif(out_gif, out_gif, xy=('30%', '5%'),
                       text_sequence='GPM: Global Precipitation Measurement (GPM) v6',
                       font_color='white', font_size = 15, duration=70)

In [3]:
geemap.add_text_to_gif(out_gif, out_gif, xy=('75%', '95%'),
                       text_sequence='Procesado por: Erik Seras',
                       font_size = 15, font_color="white",
                       duration=70)

In [5]:
geemap.show_image(out_gif)